In [1]:
# セル 1: 必要なライブラリのインポート
import os
from dotenv import load_dotenv

# Qdrant (ベクトルDB)
import qdrant_client
from llama_index.vector_stores.qdrant import QdrantVectorStore

# LlamaIndex コアコンポーネント
from llama_index.core import (
    VectorStoreIndex,
    SimpleDirectoryReader,
    StorageContext,
    Settings,
)
from llama_index.core.node_parser import CodeSplitter

# LLM & Embedding (★変更点)
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.huggingface import HuggingFaceEmbedding # ★HuggingFaceをインポート

# Git (リポジトリクローン用)
from git import Repo
import shutil # フォルダ削除用

print("ライブラリのインポート完了")

/mnt/c/Users/reore/Projects/github-repo-rag/fastapi-backend/.venv/lib/python3.10/site-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'validate_default' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'validate_default' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(


ライブラリのインポート完了


In [2]:
# セル 2: APIキーの読み込みとLLM・Embeddingの設定 (★ここが変更点)
# .env ファイルから環境変数を読み込む
load_dotenv()

# 1. LLM (回答生成用モデル) は OpenAI のまま
Settings.llm = OpenAI(model="gpt-4o")

# 2. Embedding (ベクトル化用モデル) をローカルモデルに変更
# これによりレートリミットがなくなります
# (初回実行時にモデルのダウンロードが走ります)
Settings.embedding = HuggingFaceEmbedding(
    model_name="BAAI/bge-small-en-v1.5"
)

print("LLM(OpenAI) と Embedding(Local HuggingFace) の設定完了")

LLM(OpenAI) と Embedding(Local HuggingFace) の設定完了


In [3]:
# セル 3: 定数の設定 (リポジトリとDB)

# 1. 分析対象のGitHubリポジトリ
REPO_URL = "https://github.com/tiangolo/fastapi"
# クローン先の一時ディレクトリ
REPO_PATH = "./temp_repo"
# リポジトリ内で読み込む対象（今回はチュートリアル部分のみ）
TARGET_DIR = os.path.join(REPO_PATH, "docs/en/docs/tutorial")

# 2. Qdrant の設定
QDRANT_HOST = "localhost"
QDRANT_PORT = 6333
COLLECTION_NAME = "fastapi_tutorial_local" # ★名前を変更（任意）

print(f"対象リポジトリ: {REPO_URL}")
print(f"対象DBコレクション: {COLLECTION_NAME}")

対象リポジトリ: https://github.com/tiangolo/fastapi
対象DBコレクション: fastapi_tutorial_local


In [4]:
# セル 4: GitHubリポジトリのクローン
# (注意: 実行すると ./temp_repo フォルダが作成されます)

# 既に存在する場合は一度削除
if os.path.exists(REPO_PATH):
    print(f"{REPO_PATH} を削除しています...")
    shutil.rmtree(REPO_PATH, ignore_errors=True)

print(f"{REPO_URL} を {REPO_PATH} にクローンしています...")
Repo.clone_from(REPO_URL, REPO_PATH)

print("クローン完了")

./temp_repo を削除しています...
https://github.com/tiangolo/fastapi を ./temp_repo にクローンしています...
クローン完了


In [5]:
# セル 5: データの読み込み (LlamaIndex Reader)

reader = SimpleDirectoryReader(
    input_dir=TARGET_DIR,
    required_exts=[".md"], # 今回はマークダウン(ドキュメント)のみ対象
    recursive=True, 
)

documents = reader.load_data()

print(f"{len(documents)} 個のドキュメントを読み込みました。")

48 個のドキュメントを読み込みました。


In [6]:
# セル 6: チャンキング (CodeSplitterによる分割)

splitter = CodeSplitter(
    language="markdown",
    chunk_lines=50,       
    chunk_lines_overlap=15, 
    max_chars=2000,       
)

nodes = splitter.get_nodes_from_documents(documents)

print(f"{len(nodes)} 個のノード（チャンク）に分割しました。")

250 個のノード（チャンク）に分割しました。


In [7]:
# セル 7: Qdrantクライアントの初期化とインデックス構築

# 1. Qdrantクライアントを初期化 (ローカルDBへ接続)
client = qdrant_client.QdrantClient(host=QDRANT_HOST, port=QDRANT_PORT)

# 2. LlamaIndex用のベクトルストアアダプタを作成
vector_store = QdrantVectorStore(
    client=client, 
    collection_name=COLLECTION_NAME
)

# 3. ストレージコンテキストを作成
storage_context = StorageContext.from_defaults(vector_store=vector_store)

print("Qdrantへの接続とストレージ設定完了")

# 4. インデックスの構築
# (この処理で、ローカルモデルが起動し Embedding 化され Qdrant に保存されます)
print("インデックスを構築中... (初回はモデルダウンロードのため時間がかかります)")
index = VectorStoreIndex(
    nodes, # Step 6 で作成したノード
    storage_context=storage_context,
    # Settings (LLMとEmbedding) は Step 2 で設定済み
)

print("インデックス構築完了！")

2025-10-20 16:10:43,767 - INFO - HTTP Request: GET http://localhost:6333 "HTTP/1.1 200 OK"
2025-10-20 16:10:43,771 - INFO - HTTP Request: GET http://localhost:6333/collections/fastapi_tutorial_local/exists "HTTP/1.1 200 OK"


Qdrantへの接続とストレージ設定完了
インデックスを構築中... (初回はモデルダウンロードのため時間がかかります)


2025-10-20 16:10:49,416 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-10-20 16:10:50,141 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-10-20 16:10:50,437 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 429 Too Many Requests"
2025-10-20 16:10:50,438 - INFO - Retrying request to /embeddings in 1.064000 seconds
2025-10-20 16:10:51,972 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 429 Too Many Requests"
2025-10-20 16:10:51,973 - INFO - Retrying request to /embeddings in 26.611000 seconds
2025-10-20 16:11:19,844 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 429 Too Many Requests"
2025-10-20 16:11:19,845 - INFO - Retrying request to /embeddings in 26.611000 seconds
2025-10-20 16:11:47,461 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-10-20 16:11:48,692 - INFO - HTTP Request: PUT http:

インデックス構築完了！


In [8]:
# セル 8: クエリの実行 (RAGのテスト)

# Qdrantからインデックスを読み込む
client = qdrant_client.QdrantClient(host=QDRANT_HOST, port=QDRANT_PORT)
vector_store = QdrantVectorStore(client=client, collection_name=COLLECTION_NAME)

index = VectorStoreIndex.from_vector_store(
    vector_store=vector_store,
    # Settings は Step 2 で設定済み
)

# クエリエンジンを作成
query_engine = index.as_query_engine(
    similarity_top_k=5 
)

print("クエリエンジン作成完了。質問待機中...")

2025-10-20 16:11:50,839 - INFO - HTTP Request: GET http://localhost:6333 "HTTP/1.1 200 OK"
2025-10-20 16:11:50,843 - INFO - HTTP Request: GET http://localhost:6333/collections/fastapi_tutorial_local/exists "HTTP/1.1 200 OK"
2025-10-20 16:11:50,848 - INFO - HTTP Request: GET http://localhost:6333/collections/fastapi_tutorial_local "HTTP/1.1 200 OK"


クエリエンジン作成完了。質問待機中...


In [9]:
# セル 9: 質問してみる

query = "FastAPIでパスパラメータを定義する方法は？"
# query = "Pydanticモデルとは何ですか？"

print(f"質問: {query}\n")

response = query_engine.query(query)

print("回答:")
print(str(response))

質問: FastAPIでパスパラメータを定義する方法は？



2025-10-20 16:11:51,258 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-10-20 16:11:51,268 - INFO - HTTP Request: POST http://localhost:6333/collections/fastapi_tutorial_local/points/search "HTTP/1.1 200 OK"
2025-10-20 16:11:55,054 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


回答:
FastAPIでパスパラメータを定義するには、Pythonのフォーマット文字列と同じ構文を使用します。関数の引数としてパスパラメータを宣言し、Pythonの型アノテーションを使用してその型を指定することもできます。これにより、エディタのサポートや自動的なリクエストのパースが可能になります。また、Starletteの内部ツールを使用して、パスパラメータにパスを含めることもできます。


In [10]:
# セル 10: (おまけ) ソースノードの確認

for i, node in enumerate(response.source_nodes):
    print(f"--- ソース {i+1} (Score: {node.score:.4f}) ---")
    print(node.get_content()[:500] + "...") 
    print("-" * 20)

--- ソース 1 (Score: 0.7878) ---
# Path Parameters { #path-parameters }

You can declare path "parameters" or "variables" with the same syntax used by Python format strings:

{* ../../docs_src/path_params/tutorial001.py hl[6:7] *}

The value of the path parameter `item_id` will be passed to your function as the argument `item_id`.

So, if you run this example and go to <a href="http://127.0.0.1:8000/items/foo" class="external-link" target="_blank">http://127.0.0.1:8000/items/foo</a>, you will see a response of:

```JSON
{"item_...
--------------------
--- ソース 2 (Score: 0.7825) ---
## Path parameters containing paths { #path-parameters-containing-paths }

Let's say you have a *path operation* with a path `/files/{file_path}`.

But you need `file_path` itself to contain a *path*, like `home/johndoe/myfile.txt`.

So, the URL for that file would be something like: `/files/home/johndoe/myfile.txt`.

### OpenAPI support { #openapi-support }

OpenAPI doesn't support a way to declare a *path para